# Solving Hanoi Tower with AI

The read of the illusion of thinking (from Apple's research) found reasonance in my own fun in problem solving. At a certain problem complexity, I give up when not equipped with the right tools. Perhaps then my problem solving is not much about thinking about the problem, but thinking about the tools to solve the problem. So it lead me to this question:

- Does non thinking AI solves puzzle better than thinking one when provided with the right tools?
- Does thinking AI are able to pick up the right tools to solve it?

In this notebook, I wanted to explore the first question based on reading about mcp (link to medium), hanoi algorithm (link to medium), and the Illusion of thinking article (link to article)


In this experiment, we'll:
1. Set up an MCP (Model Context Protocol) server for puzzle validation
2. Configure an AI agent to solve the Tower of Hanoi puzzle
3. Compare different approaches (with/without MCP, with/without pseudocode)
4. Look at the success rate of the AI agent

The Tower of Hanoi serves as an excellent test case because:
- It has a clear, well-defined solution
- It requires systematic thinking and planning
- It can be validated step-by-step

## Hanoi MCP server

The server provides a hanoi tower puzzle solver, a python version of the following pseudo code algorithm

```
ALGORITHM Solve(n, source, target, auxiliary, moves)
    // n = number of disks to move
    // source = starting peg (0, 1, or 2)
    // target = destination peg (0, 1, or 2)
    // auxiliary = the unused peg (0, 1, or 2)
    // moves = list to store the sequence of moves

    IF n equals 1 THEN
        // Get the top disk from source peg
        disk = the top disk on the source peg
        // Add the move to our list: [disk_id, source, target]
        ADD [disk, source, target] to moves
        RETURN
    END IF

    // Move n-1 disks from source to auxiliary peg
    Solve(n-1, source, auxiliary, target, moves)

    // Move the nth disk from source to target
    disk = the top disk on the source peg
    ADD [disk, source, target] to moves

    // Move n-1 disks from auxiliary to target
    Solve(n-1, auxiliary, target, source, moves)

    END ALGORITHM
```

In [1]:
import multiprocessing
from server.hanoi import run_mcp_server

multiprocessing.Process(target=run_mcp_server).start() 

## Example 

In [2]:
from config.hanoi_config import HanoiConfig, HanoiSolution, HanoiMove
from client.client_hanoi_tower import run_agent
from itertools import product
import pandas as pd
from datetime import datetime
import os
from tqdm import tqdm
import pickle

config = HanoiConfig(n_disks = 2)
config.use_mcp = True
config.add_pseudocode = True
config.mcp_version = 1
config.model_name = 'o3-mini'# 'gpt-4o-mini'
config.server_command = "python"
config.server_args = ["server/hanoi.py"]

[06/21/25 15:32:56] INFO     Starting Hanoi MCP server               ]8;id=271691;file:///Users/olivierbertrand/my-test-project/ai-hanoi-mcp/server/hanoi.py\hanoi.py]8;;\:]8;id=846654;file:///Users/olivierbertrand/my-test-project/ai-hanoi-mcp/server/hanoi.py#69\69]8;;\


In [3]:
# Use await instead of asyncio.run() in Jupyter notebooks
result = await run_agent(config=config)

[06/21/25 15:33:02] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=453505;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=722388;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/21/25 15:33:11] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=927572;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=676879;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/21/25 15:33:18] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=74455;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=682446;file:///Users/olivierbertrand/gcp_projects/bigquery-dbt/bigquery-dbt/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

In [4]:
if "structured_response" in result:
        print("\nStructured solution:")
        solution = result["structured_response"]
        print(f"Total moves: {solution.total_moves}")
        valid_solution = solution.validate_solution(config.n_disks)
        if not valid_solution['is_valid']:
                print(f"Invalid solution: {valid_solution}")
        else:
                print("Valid solution")
else:
        print('No structured solution')


Structured solution:
Total moves: 3
Valid solution


In [5]:
result


{'messages': [HumanMessage(content='\n    I have a puzzle with 2 disks of different sizes with\n    Initial configuration:\n    • Peg 0: 2 (bottom), ... 2, 1 (top)\n    • Peg 1: (empty)\n    • Peg 2: (empty)\n    Goal configuration:\n    • Peg 0: (empty)\n    • Peg 1: (empty)\n    • Peg 2: 2 (bottom), ... 2, 1 (top)\n    Rules:\n    • Only one disk can be moved at a time.\n    • Only the top disk from any stack can be moved.\n    • A larger disk may not be placed on top of a smaller disk.\n    Find the sequence of moves to transform the initial configuration into the goal configuration.\n    ', additional_kwargs={}, response_metadata={}, id='0b7177a7-baca-43d5-8581-698ab42ace38'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UiNiyC3lgQMsVVUD7em2SStU', 'function': {'arguments': '{"n": 2}', 'name': 'hanoi_solver'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 286, 'prompt_tokens': 1007, 'total_tokens': 1293, 'co

## Experiment

In [6]:
mcp_version = 1
saving_result_file = f'data/hanoi_results_v{mcp_version}.csv'
if os.path.exists(saving_result_file):
    completed_results = pd.read_csv(saving_result_file)
    completed_results = completed_results.loc[
        :, ['model_name', 'use_mcp', 'add_pseudocode', 'n_disks', 'ith_try']
    ]
else:
    completed_results = None


In [7]:
if mcp_version == 1:
    disks = [10, 8, 6, 4, 3, 2]
else:
    disks = [10, 8, 6]
tries = range(15)   
models = ['o4-mini', 'gpt-4.1-mini']
if mcp_version == 1:
    helpers = [
        dict(use_mcp = False, add_pseudocode = False),
        dict(use_mcp = False, add_pseudocode = True),
        dict(use_mcp = True, add_pseudocode = False)
    ]
else:
    helpers = [
        dict(use_mcp = True, add_pseudocode = False)
    ]

In [8]:
configs = []
for n_disk, model, ith_try, helper in product(disks, models, tries, helpers):
    configs.append(dict(n_disks = n_disk, model_name = model, ith_try = ith_try, use_mcp = helper['use_mcp'], add_pseudocode = helper['add_pseudocode']))
configs = pd.DataFrame(configs)
configs.set_index(['n_disks', 'model_name', 'ith_try', 'use_mcp', 'add_pseudocode'], inplace=True)
completed_results.set_index(['n_disks', 'model_name', 'ith_try', 'use_mcp', 'add_pseudocode'], inplace=True)
configs = configs.loc[~configs.index.isin(completed_results.index)]
configs.reset_index(inplace=True)

In [9]:



for _, config_series in tqdm(configs.iterrows(), total=len(configs)):
    config = HanoiConfig(n_disks = config_series.n_disks)
    config.use_mcp = config_series.use_mcp
    config.add_pseudocode = config_series.add_pseudocode
    config.model_name = config_series.model_name
    config.mcp_version = mcp_version
    config.server_command = "python"
    config.server_args = ["server/hanoi.py"]

    run_start = datetime.now()
    result = await run_agent(config)
    run_end = datetime.now()
    # To identify with the meta parameters
    result['run_start'] = run_start
    result['run_end'] = run_end
    with open('data/hanoi_results.pkl', 'ab') as f:
        pickle.dump(result, f)
    has_structured_response = "structured_response" in result
    if has_structured_response:
        solution = result["structured_response"]
        valid_solution = solution.validate_solution(config.n_disks)['is_valid']
    else:
        valid_solution = False

    to_save = pd.DataFrame(
        [dict(
            model_name = config.model_name,
            use_mcp = config.use_mcp,
            add_pseudocode = config.add_pseudocode,
            n_disks = config.n_disks,
            ith_try = config_series.ith_try,
            has_structured_response = has_structured_response,
            valid_solution = valid_solution,
            total_moves = solution.total_moves if has_structured_response else None,
            run_start = run_start,
            run_end = run_end
        )]
    )
    to_save.to_csv(saving_result_file, mode='a', header=not os.path.exists(saving_result_file))

0it [00:00, ?it/s]


# Results

## Version 1: The tool return a list of moves

Let's have a look at the results. We want to:
- Compare the thinking and non-thinking model success rate for the three help level:
   1. No MCP (i.e. no tool), No Algorithm
   2. No MCP, but with Algorithm
   3. With MCP, but without Algorithm

In [10]:
mcp_version = 1
results = pd.read_csv(f'data/hanoi_results_v{mcp_version}.csv')
results.set_index(['n_disks', 'model_name', 'use_mcp', 'add_pseudocode', 'ith_try'], inplace=True)

In [11]:
def calculate_success_rate(results):
    # Calculate success rate for each configuration
    success_rates = results.groupby(['model_name', 'use_mcp', 'add_pseudocode', 'n_disks']).agg({
        'valid_solution': ['count', 'sum', lambda x: (x.sum() / x.count() * 100).round(2)]
    }).round(2)

    # Rename columns for clarity
    success_rates.columns = ['total_attempts', 'successful_attempts', 'success_rate_percent']
    success_rates = success_rates.reset_index()
    return success_rates

success_rates = calculate_success_rate(results)

In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots for the three different configurations
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Tool ✗, Pseudocode ✗', 'Tool ✗, Pseudocode ✓', 'Tool ✓, Pseudocode ✗'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]]
)

# Define the three configurations
configs = [
    {'use_mcp': False, 'add_pseudocode': False},
    {'use_mcp': False, 'add_pseudocode': True},
    {'use_mcp': True, 'add_pseudocode': False}
]

# Colors for the models
colors = {'o4-mini': 'blue', 'gpt-4.1-mini': 'red'}

for col, config in enumerate(configs, 1):
    # Filter data for this configuration
    mask = (success_rates['use_mcp'] == config['use_mcp']) & \
           (success_rates['add_pseudocode'] == config['add_pseudocode'])
    config_data = success_rates[mask]
    
    # Plot each model
    for model in ['o4-mini', 'gpt-4.1-mini']:
        model_data = config_data[config_data['model_name'] == model]
        
        fig.add_trace(
            go.Scatter(
                x=model_data['n_disks'],
                y=model_data['success_rate_percent'],
                mode='lines+markers',
                name=f'{model}',
                line=dict(color=colors[model]),
                showlegend=(col == 1),  # Only show legend for first subplot
                hovertemplate='<b>%{fullData.name}</b><br>' +
                            'Disks: %{x}<br>' +
                            'Success Rate: %{y:.1f}%<br>' +
                            '<extra></extra>'
            ),
            row=1, col=col
        )

# Update layout
fig.update_layout(
    title='Hanoi Tower Success Rates by Configuration',
    height=500,
    width=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update axes labels
for i in range(1, 4):
    fig.update_xaxes(title_text="Number of Disks", row=1, col=i)
    fig.update_yaxes(title_text="Success Rate (%)", row=1, col=i)

fig.show()


Figure: Success rates for solving the Tower of Hanoi puzzle across different configurations and models. 
The plot shows three subplots representing different experimental conditions: 
(1) No MCP, No Pseudocode; (2) No MCP, With Pseudocode; and (3) With MCP, No Pseudocode. 
Each subplot displays success rates as a function of the number of disks (3-10) for two AI models: 
o4-mini (blue) and gpt-4.1-mini (red). The trends suggest that both models struggle more with larger disk counts, 
particularly in configurations without MCP assistance. Interestingly tool usage benefits most the non thinking models. However all models failed for 8+ disks.

> The results above seems to point out that the models struggle with correctly the tool output. The model output was only a list in the form: `content='["1", "0", "1", "2", "0", "2", "1", "1", "2"]'`. Let's try with a more verbose output.

## Version 2: The tool returns a list of moves with variables names

Let's hypothesise that the more structured the output of a tool is, the easier the processing of such an output is for LLM and LRM.

In the previous version the tools returned:
`["1", "0", "1", "2", "0", "2", "1", "1", "2"`

In the current version the tool returns:
```
["{
  \\"move_nb\\": 0
  \\"disk\\": 1
  \\"from_peg\\": 0
  \\"to_peg\\": 1
}", "
  \\"move_nb\\": 1
  \\"disk\\": 2
  \\"from_peg\\": 0
  \\"to_peg\\": 2
}", "
  \\"move_nb\\": 2
  \\"disk\\": 1
  \\"from_peg\\": 1
  \\"to_peg\\": 2
}"]'
```


In [13]:
mcp_version = 2
results = pd.read_csv(f'data/hanoi_results_v{mcp_version}.csv')
results.set_index(['n_disks', 'model_name', 'use_mcp', 'add_pseudocode', 'ith_try'], inplace=True)
success_rates = calculate_success_rate(results)

In [14]:
success_rates

,model_name,use_mcp,add_pseudocode,n_disks,total_attempts,successful_attempts,success_rate_percent
0,gpt-4.1-mini,True,False,6,15,15,100.0
1,gpt-4.1-mini,True,False,8,15,0,0.0
2,gpt-4.1-mini,True,False,10,15,0,0.0
3,o4-mini,True,False,6,15,15,100.0
4,o4-mini,True,False,8,15,0,0.0
5,o4-mini,True,False,10,15,0,0.0


In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots for the three different configurations
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=('Tool ✓, Pseudocode ✗',)
)

# Define the three configurations
configs = [
    {'use_mcp': True, 'add_pseudocode': False}
]

# Colors for the models
colors = {'o4-mini': 'blue', 'gpt-4.1-mini': 'red'}

for col, config in enumerate(configs, 1):
    # Filter data for this configuration
    mask = (success_rates['use_mcp'] == config['use_mcp']) & \
           (success_rates['add_pseudocode'] == config['add_pseudocode'])
    config_data = success_rates[mask]
    
    # Plot each model
    for model in ['o4-mini', 'gpt-4.1-mini']:
        model_data = config_data[config_data['model_name'] == model]
        
        fig.add_trace(
            go.Scatter(
                x=model_data['n_disks'],
                y=model_data['success_rate_percent'],
                mode='lines+markers',
                name=f'{model}',
                line=dict(color=colors[model]),
                showlegend=(col == 1),  # Only show legend for first subplot
                hovertemplate='<b>%{fullData.name}</b><br>' +
                            'Disks: %{x}<br>' +
                            'Success Rate: %{y:.1f}%<br>' +
                            '<extra></extra>'
            ),
            row=1, col=col
        )

# Update layout
fig.update_layout(
    title='Hanoi Tower Success Rates by Configuration',
    height=500,
    width=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update axes labels
for i in range(1, 4):
    fig.update_xaxes(title_text="Number of Disks", row=1, col=i)
    fig.update_yaxes(title_text="Success Rate (%)", row=1, col=i)

fig.show()

Figure: Success rates for solving the Tower of Hanoi puzzle with verbose tool use (i.e. v2). 
The two AI models previously used as color-coded as in previous figure: o4-mini (blue) and gpt-4.1-mini (red). The trends suggest that both models struggle more with larger disk counts. Interestingly the verbose tool improve the performance for n=6, but not the one above.

> The verbose tool improve the performance for n=6, but not the one above.

# Conclusion and open questions


## Summary of Findings

Our experimental analysis of AI models solving the Tower of Hanoi puzzle revealed:

### Performance Patterns
- **Tool Usage Impact**: 
    1. Tool usage improved more the performance of the non-thinking model that its thinking alternative
    2. The verbose tool implementation (v2) showed improved performance for 6-disk puzzles but did not extend to larger disk counts

- **Complexity Challenge**: Success rates dropped dramatically reaching 0% for 8+ disks, even when tool were used to solve the problem

### Following thoughts

1. **Performance Degradation**: Why do both models exhibit such a dramatic drop in performance between 6 and 8 disks? This suggests a fundamental limitation in the models' reasoning capabilities for this type of recursive problem.

2. **Tool Effectiveness**: While verbose tool use improved 6-disk performance, why does this improvement not scale to larger disk counts? Perhaps a more interactive approach with validation procedures would help to solve the problem accurately -> aka more tools.


# Meta information:
- Budget: 22$
- Writing Time (code+text): ~4 hours
- Running Time: ~24 hours